# NYC Taxi Data Regression Model
This is an [Azure Machine Learning Pipelines](https://aka.ms/aml-pipelines) version of two-part tutorial ([Part 1](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-data-prep), [Part 2](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-auto-train-models)) available for Azure Machine Learning.

You can combine the two part tutorial into one using AzureML Pipelines as Pipelines provide a way to stitch together various steps involved (like data preparation and training in this case) in a machine learning workflow.

In this notebook, you learn how to prepare data for regression modeling by using the [Azure Machine Learning Data Prep SDK](https://aka.ms/data-prep-sdk) for Python. You run various transformations to filter and combine two different NYC taxi data sets. Once you prepare the NYC taxi data for regression modeling, then you will use [AutoMLStep](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlstep?view=azure-ml-py) available with [Azure Machine Learning Pipelines](https://aka.ms/aml-pipelines) to define your machine learning goals and constraints as well as to launch the automated machine learning process. The automated machine learning technique iterates over many combinations of algorithms and hyperparameters until it finds the best model based on your criterion.

After you complete building the model, you can predict the cost of a taxi trip by training a model on data features. These features include the pickup day and time, the number of passengers, and the pickup location.

## Prerequisite
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at https://github.com/Azure/MachineLearningNotebooks first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc.

We will run various transformations to filter and combine two different NYC taxi data sets. We will use DataPrep SDK for this preparing data. 

Perform `pip install azureml-dataprep` if you have't already done so.

## Prepare data for regression modeling
First, we will prepare data for regression modeling. We will leverage the convenience of Azure Open Datasets along with the power of Azure Machine Learning service to create a regression model to predict NYC taxi fare prices. Perform `pip install azureml-opendatasets` to get the open dataset package.  The Open Datasets package contains a class representing each data source (NycTlcGreen and NycTlcYellow) to easily filter date parameters before downloading.


### Load data
Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid MemoryError with large datasets. To download a year of taxi data, iteratively fetch one month at a time, and before appending it to green_df_raw, randomly sample 500 records from each month to avoid bloating the dataframe. Then preview the data. To keep this process short, we are sampling data of only 1 month.

Note: Open Datasets has mirroring classes for working in Spark environments where data size and memory aren't a concern.

In [1]:
import azureml.core
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.53


In [7]:
import os
import pandas as pd

In [5]:
# from azureml.opendatasets import NycTlcGreen, NycTlcYellow

# from datetime import datetime
# from dateutil.relativedelta import relativedelta

# green_df_raw = pd.DataFrame([])
# start = datetime.strptime("1/1/2016","%m/%d/%Y")
# end = datetime.strptime("1/31/2016","%m/%d/%Y")

# number_of_months = 1
# sample_size = 5000

# for sample_month in range(number_of_months):
#     temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
#         .to_pandas_dataframe()
#     green_df_raw = green_df_raw.append(temp_df_green.sample(sample_size))

In [6]:
# yellow_df_raw = pd.DataFrame([])
# start = datetime.strptime("1/1/2016","%m/%d/%Y")
# end = datetime.strptime("1/31/2016","%m/%d/%Y")

# sample_size = 500

# for sample_month in range(number_of_months):
#     temp_df_yellow = NycTlcYellow(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
#         .to_pandas_dataframe()
#     yellow_df_raw = yellow_df_raw.append(temp_df_yellow.sample(sample_size))

### See the data

In [2]:
# import azureml.dataprep as dprep
# from IPython.display import display

# display(green_df_raw.head(5))
# display(yellow_df_raw.head(5))

NameError: name 'green_df_raw' is not defined

### Download data locally and then upload to Azure Blob
This is a one-time process to save the dave in the default datastore. 

In [6]:
# dataDir = "data"

# if not os.path.exists(dataDir):
#     os.mkdir(dataDir)

# greenDir = dataDir + "/green"
# yelloDir = dataDir + "/yellow"

# if not os.path.exists(greenDir):
#     os.mkdir(greenDir)
    
# if not os.path.exists(yelloDir):
#     os.mkdir(yelloDir)
    
# greenTaxiData = greenDir + "/part-00000"
# yellowTaxiData = yelloDir + "/part-00000"

# green_df_raw.to_csv(greenTaxiData, index=False)
# yellow_df_raw.to_csv(yellowTaxiData, index=False)

# print("Data written to local folder.")

Data written to local folder.


In [8]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.datastore import Datastore


auth = InteractiveLoginAuthentication(tenant_id="cf36141c-ddd7-45a7-b073-111f66d0b30c")
ws = Workspace.from_config()
print("Workspace: " + ws.name, "Region: " + ws.location, sep = '\n')

# Default datastore
default_store = Datastore.get(ws, datastore_name='deal_input_blob')

Workspace: avadevitsmlsvc
Region: westus2


In [9]:
# default_store.upload_files([greenTaxiData], 
#                            target_path = 'green', 
#                            overwrite = False, 
#                            show_progress = True)

# default_store.upload_files([yellowTaxiData], 
#                            target_path = 'yellow', 
#                            overwrite = False, 
#                            show_progress = True)

# print("Upload calls completed.")

### Setup Compute
#### Create new or use an existing compute

In [10]:
from pygit2 import Repository
from azureml.core.compute import AmlCompute, ComputeTarget, DataFactoryCompute

def get_or_create_compute(workspace, compute_target_name, **kwargs):
    compute_target = ComputeTarget.create(
            workspace,
            compute_target_name,
            AmlCompute.provisioning_configuration(**kwargs))

    compute_target.wait_for_completion(show_output=True)
    print(compute_target_name, "has been created")
    return compute_target

def make_resource_name(prefix, suffix, max_len, sep='-'):
    '''
    project max_len:36, compute max_len:16
    '''
    # suffix will be abbreviated if it is longer than
    # (max_length - len(prefix) - len(sep))
    suffix_max_len = min(len(suffix), max_len - len(prefix) - len(sep))
    suffix_abbv = (suffix
                   [0:suffix_max_len]
                   .replace('_', '-')
                   )
    resource_name = sep.join([prefix, suffix_abbv])
    return resource_name

prefix = 'deal'
repo_name = Repository('../../..').head.shorthand
compute_target_name = make_resource_name(prefix, repo_name, max_len=16)

aml_compute = get_or_create_compute(workspace=ws,
                                       compute_target_name=compute_target_name,
                                       vm_size='STANDARD_D2_V2',
                                       max_nodes=8
                                       )
aml_compute

Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
deal-master has been created


AmlCompute(workspace=Workspace.create(name='avadevitsmlsvc', subscription_id='ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e', resource_group='RG-ITSMLTeam-Dev'), name=deal-master, id=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourceGroups/RG-ITSMLTeam-Dev/providers/Microsoft.MachineLearningServices/workspaces/avadevitsmlsvc/computes/deal-master, type=AmlCompute, provisioning_state=Succeeded, location=westus2, tags=None)

#### Define RunConfig for the compute
We need `azureml-dataprep` SDK for all the steps below. We will also use `pandas`, `scikit-learn` and `automl` for the training step. Defining the `runconfig` for that.

In [11]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Create a new runconfig object
aml_run_config = RunConfiguration()

# Use the aml_compute you created above. 
aml_run_config.target = aml_compute

# Enable Docker
aml_run_config.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
aml_run_config.environment.docker.base_image = "mcr.microsoft.com/azureml/base:0.2.1"

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
aml_run_config.environment.python.user_managed_dependencies = False

# Auto-prepare the Docker image when used for execution (if it is not already prepared)
aml_run_config.auto_prepare_environment = True

# Specify CondaDependencies obj, add necessary packages
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn'], 
    pip_packages=['azureml-sdk', 'azureml-dataprep', 'azureml-train-automl==1.0.33'], 
    pin_sdk_version=False)

print ("Run configuration created.")

WARNING - 'auto_prepare_environment' is deprecated and unused. It will be removed in a future release.
WARNING - 'auto_prepare_environment' is deprecated and unused. It will be removed in a future release.


Run configuration created.


### Prepare data
Now we will prepare for regression modeling by using the `Azure Machine Learning Data Prep SDK for Python`. We run various transformations to filter and combine two different NYC taxi data sets.

We achieve this by creating a separate step for each transformation as this allows us to reuse the steps and saves us from running all over again in case of any change. We will keep data preparation scripts in one subfolder and training scripts in another.

> The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

#### Define Useful Colums
Here we are defining a set of "useful" columns for both Green and Yellow taxi data.

In [13]:
# display(green_df_raw.columns)
# display(yellow_df_raw.columns)

# useful columns needed for the Azure Machine Learning NYC Taxi tutorial
useful_columns = str(["cost", "distance", "dropoff_datetime", "dropoff_latitude", 
                      "dropoff_longitude", "passengers", "pickup_datetime", 
                      "pickup_latitude", "pickup_longitude", "store_forward", "vendor"]).replace(",", ";")

print("Useful columns defined.")

Useful columns defined.


#### Cleanse Green taxi data

In [14]:
from azureml.data.data_reference import DataReference 
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

# python scripts folder
prepare_data_folder = './scripts/prepdata'

blob_green_data = DataReference(
    datastore=default_store,
    data_reference_name="green_taxi_data",
    path_on_datastore="green/part-00000")

# rename columns as per Azure Machine Learning NYC Taxi tutorial
green_columns = str({ 
    "vendorID": "vendor",
    "lpepPickupDatetime": "pickup_datetime",
    "lpepDropoffDatetime": "dropoff_datetime",
    "storeAndFwdFlag": "store_forward",
    "pickupLongitude": "pickup_longitude",
    "pickupLatitude": "pickup_latitude",
    "dropoffLongitude": "dropoff_longitude",
    "dropoffLatitude": "dropoff_latitude",
    "passengerCount": "passengers",
    "fareAmount": "cost",
    "tripDistance": "distance"
}).replace(",", ";")

# Define output after cleansing step
cleansed_green_data = PipelineData("green_taxi_data", datastore=default_store)

print('Cleanse script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# cleansing step creation
# See the cleanse.py for details about input and output
cleansingStepGreen = PythonScriptStep(
    name="Cleanse Green Taxi Data",
    script_name="cleanse.py", 
    arguments=["--input_cleanse", blob_green_data, 
               "--useful_columns", useful_columns,
               "--columns", green_columns,
               "--output_cleanse", cleansed_green_data],
    inputs=[blob_green_data],
    outputs=[cleansed_green_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("cleansingStepGreen created.")

Cleanse script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\prepdata.
cleansingStepGreen created.


#### Cleanse Yellow taxi data

In [15]:
blob_yellow_data = DataReference(
    datastore=default_store,
    data_reference_name="yellow_taxi_data",
    path_on_datastore="yellow/part-00000")

yellow_columns = str({
    "vendorID": "vendor",
    "tpepPickupDateTime": "pickup_datetime",
    "tpepDropoffDateTime": "dropoff_datetime",
    "storeAndFwdFlag": "store_forward",
    "startLon": "pickup_longitude",
    "startLat": "pickup_latitude",
    "endLon": "dropoff_longitude",
    "endLat": "dropoff_latitude",
    "passengerCount": "passengers",
    "fareAmount": "cost",
    "tripDistance": "distance"
}).replace(",", ";")

# Define output after cleansing step
cleansed_yellow_data = PipelineData("yellow_taxi_data", datastore=default_store)

print('Cleanse script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# cleansing step creation
# See the cleanse.py for details about input and output
cleansingStepYellow = PythonScriptStep(
    name="Cleanse Yellow Taxi Data",
    script_name="cleanse.py", 
    arguments=["--input_cleanse", blob_yellow_data, 
               "--useful_columns", useful_columns,
               "--columns", yellow_columns,
               "--output_cleanse", cleansed_yellow_data],
    inputs=[blob_yellow_data],
    outputs=[cleansed_yellow_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("cleansingStepYellow created.")

Cleanse script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\prepdata.
cleansingStepYellow created.


#### Merge cleansed Green and Yellow datasets
We are creating a single data source by merging the cleansed versions of Green and Yellow taxi data.

In [16]:
# Define output after merging step
merged_data = PipelineData("merged_data", datastore=default_store)

print('Merge script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# merging step creation
# See the merge.py for details about input and output
mergingStep = PythonScriptStep(
    name="Merge Taxi Data",
    script_name="merge.py", 
    arguments=["--input_green_merge", cleansed_green_data, 
               "--input_yellow_merge", cleansed_yellow_data,
               "--output_merge", merged_data],
    inputs=[cleansed_green_data, cleansed_yellow_data],
    outputs=[merged_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("mergingStep created.")

Merge script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\prepdata.
mergingStep created.


#### Filter data
This step filters out coordinates for locations that are outside the city border. We use a TypeConverter object to change the latitude and longitude fields to decimal type. 

In [17]:
# Define output after merging step
filtered_data = PipelineData("filtered_data", datastore=default_store)

print('Filter script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# filter step creation
# See the filter.py for details about input and output
filterStep = PythonScriptStep(
    name="Filter Taxi Data",
    script_name="filter.py", 
    arguments=["--input_filter", merged_data, 
               "--output_filter", filtered_data],
    inputs=[merged_data],
    outputs=[filtered_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("FilterStep created.")

Filter script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\prepdata.
FilterStep created.


#### Normalize data
In this step, we split the pickup and dropoff datetime values into the respective date and time columns and then we rename the columns to use meaningful names.

In [18]:
# Define output after normalize step
normalized_data = PipelineData("normalized_data", datastore=default_store)

print('Normalize script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# normalize step creation
# See the normalize.py for details about input and output
normalizeStep = PythonScriptStep(
    name="Normalize Taxi Data",
    script_name="normalize.py", 
    arguments=["--input_normalize", filtered_data, 
               "--output_normalize", normalized_data],
    inputs=[filtered_data],
    outputs=[normalized_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("normalizeStep created.")

Normalize script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\prepdata.
normalizeStep created.


#### Transform data
Transform the normalized taxi data to final required format. This steps does the following:

- Split the pickup and dropoff date further into the day of the week, day of the month, and month values. 
- To get the day of the week value, uses the derive_column_by_example() function. The function takes an array parameter of example objects that define the input data, and the preferred output. The function automatically determines the preferred transformation. For the pickup and dropoff time columns, split the time into the hour, minute, and second by using the split_column_by_example() function with no example parameter.
- After new features are generated, use the drop_columns() function to delete the original fields as the newly generated features are preferred. 
- Rename the rest of the fields to use meaningful descriptions.

In [19]:
# Define output after transforme step
transformed_data = PipelineData("transformed_data", datastore=default_store)

print('Transform script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# transform step creation
# See the transform.py for details about input and output
transformStep = PythonScriptStep(
    name="Transform Taxi Data",
    script_name="transform.py", 
    arguments=["--input_transform", normalized_data,
               "--output_transform", transformed_data],
    inputs=[normalized_data],
    outputs=[transformed_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("transformStep created.")

Transform script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\prepdata.
transformStep created.


### Extract features
Add the following columns to be features for our model creation. The prediction value will be *cost*.

In [20]:
feature_columns = str(['pickup_weekday','pickup_hour', 'distance','passengers', 'vendor']).replace(",", ";")

train_model_folder = './scripts/trainmodel'

print('Extract script is in {}.'.format(os.path.realpath(train_model_folder)))

# features data after transform step
features_data = PipelineData("features_data", datastore=default_store)

# featurization step creation
# See the featurization.py for details about input and output
featurizationStep = PythonScriptStep(
    name="Extract Features",
    script_name="featurization.py", 
    arguments=["--input_featurization", transformed_data, 
               "--useful_columns", feature_columns,
               "--output_featurization", features_data],
    inputs=[transformed_data],
    outputs=[features_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=train_model_folder,
    allow_reuse=True
)

print("featurizationStep created.")

Extract script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\trainmodel.
featurizationStep created.


### Extract label

In [21]:
label_columns = str(['cost']).replace(",", ";")

# label data after transform step
label_data = PipelineData("label_data", datastore=default_store)

print('Extract script is in {}.'.format(os.path.realpath(train_model_folder)))

# label step creation
# See the featurization.py for details about input and output
labelStep = PythonScriptStep(
    name="Extract Labels",
    script_name="featurization.py", 
    arguments=["--input_featurization", transformed_data, 
               "--useful_columns", label_columns,
               "--output_featurization", label_data],
    inputs=[transformed_data],
    outputs=[label_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=train_model_folder,
    allow_reuse=True
)

print("labelStep created.")

Extract script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\trainmodel.
labelStep created.


### Split the data into train and test sets
This function segregates the data into the **x**, features, dataset for model training and **y**, values to predict, dataset for testing.

In [22]:
# train and test splits output
output_split_train_x = PipelineData("output_split_train_x", datastore=default_store)
output_split_train_y = PipelineData("output_split_train_y", datastore=default_store)
output_split_test_x = PipelineData("output_split_test_x", datastore=default_store)
output_split_test_y = PipelineData("output_split_test_y", datastore=default_store)

print('Data spilt script is in {}.'.format(os.path.realpath(train_model_folder)))

# test train split step creation
# See the train_test_split.py for details about input and output
testTrainSplitStep = PythonScriptStep(
    name="Train Test Data Split",
    script_name="train_test_split.py", 
    arguments=["--input_split_features", features_data, 
               "--input_split_labels", label_data,
               "--output_split_train_x", output_split_train_x,
               "--output_split_train_y", output_split_train_y,
               "--output_split_test_x", output_split_test_x,
               "--output_split_test_y", output_split_test_y],
    inputs=[features_data, label_data],
    outputs=[output_split_train_x, output_split_train_y, output_split_test_x, output_split_test_y],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=train_model_folder,
    allow_reuse=True
)

print("testTrainSplitStep created.")

Data spilt script is in c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\trainmodel.
testTrainSplitStep created.


## Use automated machine learning to build regression model
Now we will use **automated machine learning** to build the regression model. We will use [AutoMLStep](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlstep?view=azure-ml-py) in AML Pipelines for this part. These functions use various features from the data set and allow an automated model to build relationships between the features and the price of a taxi trip.

### Automatically train a model

#### Create experiment

In [23]:
from azureml.core import Experiment

experiment = Experiment(ws, 'NYCTaxi_Tutorial_Pipelines')

print("Experiment created")

Experiment created


#### Create get_data script

A script with `get_data()` function is necessary to fetch training features(X) and labels(Y) on remote compute, from input data. Here we use mounted path of `train_test_split` step to get the x and y train values. They are added as environment variable on compute machine by default

Note: Every DataReference are added as environment variable on compute machine since the defualt mode is mount

In [24]:
print('get_data.py will be written to {}.'.format(os.path.realpath(train_model_folder)))

get_data.py will be written to c:\Users\anders.swanson\Documents\MachineLearningNotebooks-1\how-to-use-azureml\machine-learning-pipelines\nyc-taxi-data-regression-model-building\scripts\trainmodel.


In [25]:
# %%writefile $train_model_folder/get_data.py

# def get_data():
#     print("In get_data")
#     print(os.environ['AZUREML_DATAREFERENCE_output_split_train_x'])
#     file_name="part-00000"
#     X_train = pd.read_csv(os.path.join(os.environ['AZUREML_DATAREFERENCE_output_split_train_x'],file_name), header=0)
#     y_train = pd.read_csv(os.environ['AZUREML_DATAREFERENCE_output_split_train_y'],file_name, header=0)

#     return {"X": X_train.values, "y": y_train.values.flatten()}

#### Define settings for autogeneration and tuning

Here we define the experiment parameter and model settings for autogeneration and tuning. We can specify automl_settings as **kwargs as well. Also note that we have to use a get_data() function for remote excutions. See get_data script for more details.

Use your defined training settings as a parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

Note: When using AmlCompute, we can't pass Numpy arrays directly to the fit method.

In [26]:
import logging
from azureml.train.automl import AutoMLConfig

# Change iterations to a reasonable number (50) to get better accuracy
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 2,
    "primary_metric" : 'spearman_correlation',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automated_ml_errors.log',
                             path = train_model_folder,
                             compute_target=aml_compute,
                             run_configuration=aml_run_config,
                             data_script = train_model_folder + "/get_data.py",
                             **automl_settings)
                             
print("AutoML config created.")

AutoML config created.


#### Define AutoMLStep

In [27]:
from azureml.train.automl import AutoMLStep

trainWithAutomlStep = AutoMLStep(
    name='AutoML_Regression',
    automl_config=automl_config,
    inputs=[output_split_train_x, output_split_train_y],
    allow_reuse=True,
    hash_paths=[os.path.realpath(train_model_folder)])

print("trainWithAutomlStep created.")

WARNING - Parameter 'hash_paths' is deprecated, will be removed. All files under  `path` and the `data_script` file specified in `AutoMLConfig` is hashed except files listed in .amlignore or .gitignore under `path`.


trainWithAutomlStep created.


#### Build and run the pipeline

In [32]:
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline_steps = [trainWithAutomlStep]

pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

pipeline_run = experiment.submit(pipeline, regenerate_outputs=False)

print("Pipeline submitted for execution.")

WARNING - 'auto_prepare_environment' is deprecated and unused. It will be removed in a future release.


In get_data


ValueError: Duplicate pipeline output with name _default_metrics_AutoML_Regression

In [33]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [31]:
pipeline_run.cancel()

### Explore the results

In [30]:
# Before we proceed we need to wait for the run to complete.
pipeline_run.wait_for_completion()

# functions to download output to local and fetch as dataframe
def get_download_path(download_path, output_name):
    output_folder = os.listdir(download_path + '/azureml')[0]
    path =  download_path + '/azureml/' + output_folder + '/' + output_name
    return path

def fetch_df(step, output_name):
    output_data = step.get_output_data(output_name)
    
    download_path = './outputs/' + output_name
    output_data.download(download_path)
    df_path = get_download_path(download_path, output_name) + '/part-00000'
    return dprep.auto_read_file(path=df_path)

PipelineRunId: 52379dea-49f7-46a3-bcc2-c7a230dc87af
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourceGroups/RG-ITSMLTeam-Dev/providers/Microsoft.MachineLearningServices/workspaces/avadevitsmlsvc/experiments/NYCTaxi_Tutorial_Pipelines/runs/52379dea-49f7-46a3-bcc2-c7a230dc87af
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 11c637c6-135c-4c87-a765-dabfaece38dc
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourceGroups/RG-ITSMLTeam-Dev/providers/Microsoft.MachineLearningServices/workspaces/avadevitsmlsvc/experiments/NYCTaxi_Tutorial_Pipelines/runs/11c637c6-135c-4c87-a765-dabfaece38dc

StepRun(Merge Taxi Data) Execution Summary
StepRun( Merge Taxi Data ) Status: Finished
{'runId': '11c637c6-135c-4c87-a765-dabfaece38dc', 'target': 'deal-master', 'status': 'Completed', 'startTimeUtc': '2019-07-31T01:45:12.326436Z', 'endTimeUtc': '2019-07-31T01:45:12




StepRunId: ad944606-c3d1-45c6-a4d0-3bd38d453d03
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourceGroups/RG-ITSMLTeam-Dev/providers/Microsoft.MachineLearningServices/workspaces/avadevitsmlsvc/experiments/NYCTaxi_Tutorial_Pipelines/runs/ad944606-c3d1-45c6-a4d0-3bd38d453d03

StepRun(Cleanse Yellow Taxi Data) Execution Summary
StepRun( Cleanse Yellow Taxi Data ) Status: Finished
{'runId': 'ad944606-c3d1-45c6-a4d0-3bd38d453d03', 'target': 'deal-master', 'status': 'Completed', 'startTimeUtc': '2019-07-31T01:45:11.501761Z', 'endTimeUtc': '2019-07-31T01:45:11.540869Z', 'properties': {'azureml.reusedrunid': 'a9679a56-b09b-4bf9-8722-205e935e0a3b', 'azureml.reusednodeid': '0aa2a315', 'azureml.reusedpipeline': '193137e9-1508-4a46-8986-13885c7d2870', 'azureml.reusedpipelinerunid': '193137e9-1508-4a46-8986-13885c7d2870', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '8331f101', 'ContentSnapshotId': '05e5633c-437b-4e28-b2d3-




StepRunId: 1159d4fa-5873-4ce6-b79c-3a7a69f9c2b6
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourceGroups/RG-ITSMLTeam-Dev/providers/Microsoft.MachineLearningServices/workspaces/avadevitsmlsvc/experiments/NYCTaxi_Tutorial_Pipelines/runs/1159d4fa-5873-4ce6-b79c-3a7a69f9c2b6

StepRun(Normalize Taxi Data) Execution Summary
StepRun( Normalize Taxi Data ) Status: Finished
{'runId': '1159d4fa-5873-4ce6-b79c-3a7a69f9c2b6', 'target': 'deal-master', 'status': 'Completed', 'startTimeUtc': '2019-07-31T01:45:13.258519Z', 'endTimeUtc': '2019-07-31T01:45:13.298655Z', 'properties': {'azureml.reusedrunid': '7d9c5e61-06c6-43b1-a27f-1da5bf35e823', 'azureml.reusednodeid': 'ef703ad0', 'azureml.reusedpipeline': '193137e9-1508-4a46-8986-13885c7d2870', 'azureml.reusedpipelinerunid': '193137e9-1508-4a46-8986-13885c7d2870', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '04a625df', 'ContentSnapshotId': '05e5633c-437b-4e28-b2d3-29b25ef313




StepRunId: 033785e3-9938-415a-8b9a-9e799ef7c108
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourceGroups/RG-ITSMLTeam-Dev/providers/Microsoft.MachineLearningServices/workspaces/avadevitsmlsvc/experiments/NYCTaxi_Tutorial_Pipelines/runs/033785e3-9938-415a-8b9a-9e799ef7c108
StepRun( Extract Features ) Status: Running


ExperimentExecutionException: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run

#### View cleansed taxi data

In [33]:
green_cleanse_step = pipeline_run.find_step_run(cleansingStepGreen.name)[0]
yellow_cleanse_step = pipeline_run.find_step_run(cleansingStepYellow.name)[0]

cleansed_green_df = fetch_df(green_cleanse_step, cleansed_green_data.name)
cleansed_yellow_df = fetch_df(yellow_cleanse_step, cleansed_yellow_data.name)

display(cleansed_green_df.head(5))
display(cleansed_yellow_df.head(5))

,vendor,pickup_datetime,dropoff_datetime,passengers,distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_forward,cost
0,2,2016-01-18 05:18:00,2016-01-18 05:24:26,1,2.02,-73.94,40.80,-73.94,40.82,False,8.00
1,2,2016-01-07 07:40:55,2016-01-07 08:01:44,1,5.21,-73.94,40.80,-73.86,40.82,False,19.50
2,2,2016-01-27 11:05:40,2016-01-27 11:15:15,1,1.72,-73.88,40.88,-73.90,40.86,False,8.50
3,2,2016-01-26 04:45:44,2016-01-26 04:54:44,1,0.00,-73.94,40.76,-73.94,40.76,False,6.50
4,2,2016-01-19 08:36:13,2016-01-19 09:01:14,1,8.82,-73.84,40.72,-73.98,40.74,False,26.50


,vendor,pickup_datetime,dropoff_datetime,passengers,distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_forward,cost
0,2,2016-01-19 20:20:50,2016-01-19 20:34:06,1,6.62,-73.95,40.78,-73.93,40.87,False,20.00
1,1,2016-01-18 23:53:03,2016-01-19 00:15:01,2,11.40,-73.87,40.77,-73.98,40.75,False,32.50
2,1,2016-01-09 17:19:59,2016-01-09 17:37:17,1,2.10,-73.95,40.78,-73.98,40.76,False,12.50
3,1,2016-01-09 09:26:37,2016-01-09 09:39:18,1,2.70,-73.98,40.73,-74.00,40.76,False,11.50
4,2,2016-01-08 16:30:16,2016-01-08 16:39:34,1,2.17,-73.96,40.77,-73.94,40.79,False,9.00


#### View the combined taxi data profile

In [34]:
merge_step = pipeline_run.find_step_run(mergingStep.name)[0]
combined_df = fetch_df(merge_step, merged_data.name)

display(combined_df.get_profile())

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.INTEGER,1.00,2.00,5000.00,0.00,5000.00,0.00,0.00,0.00,2,1.00,1.00,1.00,2.00,2.00,2.00,2.00,2.00,2.00,1.78,0.42,0.17,-1.33,-0.23
pickup_datetime,FieldType.DATE,2016-01-01 00:05:38+00:00,2016-01-30 23:53:28+00:00,5000.00,0.00,5000.00,0.00,0.00,0.00,,,,,,,,,,,,,,,
dropoff_datetime,FieldType.DATE,2016-01-01 00:17:03+00:00,2016-01-31 20:59:48+00:00,5000.00,0.00,5000.00,0.00,0.00,0.00,,,,,,,,,,,,,,,
passengers,FieldType.INTEGER,0.00,8.00,5000.00,0.00,5000.00,0.00,0.00,0.00,8,1.00,1.00,1.00,1.00,1.00,1.00,4.43,6.00,6.00,1.36,1.03,1.05,3.19,9.54
distance,FieldType.DECIMAL,0.00,28.51,5000.00,0.00,5000.00,0.00,0.00,0.00,918,0.00,0.63,0.62,1.03,1.85,3.49,8.42,13.94,20.13,2.78,2.78,7.74,2.53,9.21
pickup_longitude,FieldType.DECIMAL,-74.16,0.00,5000.00,0.00,5000.00,0.00,0.00,0.00,,-74.03,-73.99,-73.99,-73.96,-73.95,-73.92,-73.84,-73.81,-22.12,-73.85,2.56,6.56,28.80,827.55
pickup_latitude,FieldType.DECIMAL,0.00,40.93,5000.00,0.00,5000.00,0.00,0.00,0.00,,10.14,40.68,40.68,40.69,40.75,40.80,40.84,40.87,40.89,40.70,1.41,1.99,-28.73,825.22
dropoff_longitude,FieldType.DECIMAL,-74.18,0.00,5000.00,0.00,5000.00,0.00,0.00,0.00,,-74.10,-73.99,-73.99,-73.97,-73.95,-73.91,-73.83,-73.78,0.00,-73.77,3.46,12.00,21.24,449.18
dropoff_latitude,FieldType.DECIMAL,0.00,40.93,5000.00,0.00,5000.00,0.00,0.00,0.00,,0.00,40.67,40.67,40.70,40.75,40.79,40.84,40.87,40.91,40.66,1.91,3.65,-21.21,448.54
store_forward,FieldType.BOOLEAN,False,True,5000.00,0.00,5000.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,


#### View the filtered taxi data profile

In [35]:
filter_step = pipeline_run.find_step_run(filterStep.name)[0]
filtered_df = fetch_df(filter_step, filtered_data.name)

display(filtered_df.get_profile())

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.INTEGER,1.00,2.00,487.00,0.00,487.00,0.00,0.00,0.00,2,1.00,1.00,1.00,1.00,2.00,2.00,2.00,2.00,2.00,1.52,0.50,0.25,-0.09,-2.00
pickup_datetime,FieldType.DATE,2016-01-01 00:28:12+00:00,2016-01-30 21:55:02+00:00,487.00,0.00,487.00,0.00,0.00,0.00,487,,,,,,,,,,,,,,
dropoff_datetime,FieldType.DATE,2016-01-01 00:55:35+00:00,2016-01-30 22:01:36+00:00,487.00,0.00,487.00,0.00,0.00,0.00,487,,,,,,,,,,,,,,
passengers,FieldType.INTEGER,1.00,6.00,487.00,0.00,487.00,0.00,0.00,0.00,6,1.00,1.00,1.00,1.00,1.00,2.00,5.00,6.00,6.00,1.75,1.40,1.97,1.88,2.21
distance,FieldType.DECIMAL,0.00,22.73,487.00,0.00,487.00,0.00,0.00,0.00,241,0.00,0.62,0.60,0.94,1.65,3.00,9.88,18.79,22.73,2.81,3.58,12.84,3.03,9.77
pickup_longitude,FieldType.DECIMAL,-74.02,-73.78,487.00,0.00,487.00,0.00,0.00,0.00,470,-74.02,-74.00,-74.00,-73.99,-73.98,-73.97,-73.92,-73.78,-73.78,-73.97,0.04,0.00,3.37,13.17
pickup_latitude,FieldType.DECIMAL,40.64,40.83,487.00,0.00,487.00,0.00,0.00,0.00,479,40.64,40.73,40.73,40.74,40.75,40.77,40.79,40.80,40.83,40.75,0.03,0.00,-1.50,4.89
dropoff_longitude,FieldType.DECIMAL,-74.04,-73.78,487.00,0.00,487.00,0.00,0.00,0.00,477,-74.04,-74.00,-74.00,-73.99,-73.98,-73.96,-73.92,-73.79,-73.78,-73.97,0.03,0.00,3.19,13.71
dropoff_latitude,FieldType.DECIMAL,40.61,40.87,487.00,0.00,487.00,0.00,0.00,0.00,481,40.61,40.72,40.72,40.74,40.76,40.77,40.79,40.81,40.87,40.75,0.03,0.00,-1.07,3.60
store_forward,FieldType.BOOLEAN,False,True,487.00,0.00,487.00,0.00,0.00,0.00,2,,,,,,,,,,,,,,


#### View normalized taxi data

In [36]:
normalize_step = pipeline_run.find_step_run(normalizeStep.name)[0]
normalized_df = fetch_df(normalize_step, normalized_data.name)

display(normalized_df.head(5))

,vendor,pickup_datetime,pickup_date,pickup_time,dropoff_datetime,dropoff_date,dropoff_time,passengers,distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_forward,cost
0,2,2016-01-18 05:18:00,2016-01-18,05:18:00,2016-01-18 05:24:26,2016-01-18,05:24:26,1,2.02,-73.94,40.80,-73.94,40.82,False,8.00
1,2,2016-01-07 07:40:55,2016-01-07,07:40:55,2016-01-07 08:01:44,2016-01-07,08:01:44,1,5.21,-73.94,40.80,-73.86,40.82,False,19.50
2,2,2016-01-27 11:05:40,2016-01-27,11:05:40,2016-01-27 11:15:15,2016-01-27,11:15:15,1,1.72,-73.88,40.88,-73.90,40.86,False,8.50
3,2,2016-01-26 04:45:44,2016-01-26,04:45:44,2016-01-26 04:54:44,2016-01-26,04:54:44,1,0.00,-73.94,40.76,-73.94,40.76,False,6.50
4,2,2016-01-19 08:36:13,2016-01-19,08:36:13,2016-01-19 09:01:14,2016-01-19,09:01:14,1,8.82,-73.84,40.72,-73.98,40.74,False,26.50


#### View transformed taxi data

In [37]:
transform_step = pipeline_run.find_step_run(transformStep.name)[0]
transformed_df = fetch_df(transform_step, transformed_data.name)

display(transformed_df.get_profile())
display(transformed_df.head(5))

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.INTEGER,1.00,2.00,484.00,0.00,484.00,0.00,0.00,0.00,2,1.00,1.00,1.00,1.00,2.00,2.00,2.00,2.00,2.00,1.53,0.50,0.25,-0.11,-1.99
pickup_weekday,FieldType.STRING,Friday,Wednesday,484.00,0.00,484.00,0.00,0.00,0.00,7,,,,,,,,,,,,,,
pickup_hour,FieldType.INTEGER,0.00,23.00,484.00,0.00,484.00,0.00,0.00,0.00,24,0.00,4.00,4.00,8.64,13.89,18.00,22.00,23.00,23.00,13.14,6.28,39.46,-0.37,-0.77
pickup_minute,FieldType.INTEGER,0.00,59.00,484.00,0.00,484.00,0.00,0.00,0.00,60,0.00,5.00,5.00,14.19,30.79,44.75,57.00,59.00,59.00,29.72,17.63,310.68,-0.05,-1.24
pickup_second,FieldType.INTEGER,0.00,59.00,484.00,0.00,484.00,0.00,0.00,0.00,60,0.00,5.35,5.00,13.10,28.46,44.50,56.10,59.00,59.00,28.94,17.68,312.75,0.08,-1.28
dropoff_weekday,FieldType.STRING,Friday,Wednesday,484.00,0.00,484.00,0.00,0.00,0.00,7,,,,,,,,,,,,,,
dropoff_hour,FieldType.INTEGER,0.00,23.00,484.00,0.00,484.00,0.00,0.00,0.00,24,0.00,4.00,4.00,9.00,14.00,18.50,22.00,23.00,23.00,13.26,6.30,39.66,-0.39,-0.76
dropoff_minute,FieldType.INTEGER,0.00,59.00,484.00,0.00,484.00,0.00,0.00,0.00,60,0.00,5.00,5.00,14.00,29.89,45.75,56.00,58.66,59.00,29.55,17.70,313.30,-0.05,-1.26
dropoff_second,FieldType.INTEGER,0.00,59.00,484.00,0.00,484.00,0.00,0.00,0.00,60,0.00,5.00,5.00,14.29,28.89,42.75,56.20,59.00,59.00,28.85,17.10,292.38,0.03,-1.12
passengers,FieldType.INTEGER,1.00,6.00,484.00,0.00,484.00,0.00,0.00,0.00,6,1.00,1.00,1.00,1.00,1.00,2.00,5.00,6.00,6.00,1.75,1.41,1.97,1.87,2.17


,vendor,pickup_weekday,pickup_hour,pickup_minute,pickup_second,dropoff_weekday,dropoff_hour,dropoff_minute,dropoff_second,passengers,distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_forward,cost
0,2,Tuesday,20,20,50,Tuesday,20,34,6,1,6.62,-73.95,40.78,-73.93,40.87,False,20.00
1,1,Monday,23,53,3,Tuesday,0,15,1,2,11.40,-73.87,40.77,-73.98,40.75,False,32.50
2,1,Saturday,17,19,59,Saturday,17,37,17,1,2.10,-73.95,40.78,-73.98,40.76,False,12.50
3,1,Saturday,9,26,37,Saturday,9,39,18,1,2.70,-73.98,40.73,-74.00,40.76,False,11.50
4,2,Friday,16,30,16,Friday,16,39,34,1,2.17,-73.96,40.77,-73.94,40.79,False,9.00


#### View training data used by AutoML

In [38]:
split_step = pipeline_run.find_step_run(testTrainSplitStep.name)[0]
train_split_x = fetch_df(split_step, output_split_train_x.name)
train_split_y = fetch_df(split_step, output_split_train_y.name)

display_x_train = train_split_x.keep_columns(columns=["vendor", "pickup_weekday", "pickup_hour", "passengers", "distance"])
display_y_train = train_split_y.rename_columns(column_pairs={"Column1": "cost"})

display(display_x_train.get_profile())
display(display_x_train.head(5))
display(display_y_train.get_profile())
display(display_y_train.head(5))

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.INTEGER,1.00,2.00,4278.00,0.00,4278.00,0.00,0.00,0.00,2,1.00,1.00,1.00,1.81,2.00,2.00,2.00,2.00,2.00,1.75,0.43,0.19,-1.18,-0.62
pickup_weekday,FieldType.STRING,Friday,Wednesday,4278.00,0.00,4278.00,0.00,0.00,0.00,7,,,,,,,,,,,,,,
pickup_hour,FieldType.INTEGER,0.00,23.00,4278.00,0.00,4278.00,0.00,0.00,0.00,24,0.00,2.14,2.00,8.93,15.00,19.00,22.46,23.00,23.00,13.41,6.71,45.01,-0.48,-0.85
passengers,FieldType.INTEGER,1.00,8.00,4278.00,0.00,4278.00,0.00,0.00,0.00,7,1.00,1.00,1.00,1.00,1.00,1.00,4.95,6.00,6.00,1.40,1.08,1.16,3.01,8.28
distance,FieldType.DECIMAL,0.01,22.73,4278.00,0.00,4278.00,0.00,0.00,0.00,872,0.02,0.69,0.68,1.06,1.88,3.49,8.55,14.69,19.49,2.83,2.82,7.94,2.49,8.00


,vendor,pickup_weekday,pickup_hour,passengers,distance
0,2,Saturday,21,1,1.21
1,2,Thursday,9,1,2.01
2,2,Saturday,0,6,4.84
3,2,Sunday,19,1,2.20
4,2,Thursday,8,1,0.47


,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis
cost,FieldType.DECIMAL,2.50,240.00,4278.00,0.00,4278.00,0.00,0.00,0.00,108,2.50,5.00,5.00,6.50,9.49,14.62,29.39,47.85,63.79,12.12,9.32,86.91,5.18,87.70


,cost
0,5.50
1,14.50
2,16.50
3,11.50
4,4.50


#### View the details of the AutoML run

In [39]:
from azureml.train.automl.run import AutoMLRun
#from azureml.widgets import RunDetails

# workaround to get the automl run as its the last step in the pipeline 
# and get_steps() returns the steps from latest to first

for step in pipeline_run.get_steps():
    automl_step_run_id = step.id
    print(step.name)
    print(automl_step_run_id)
    break

automl_run = AutoMLRun(experiment = experiment, run_id=automl_step_run_id)
#RunDetails(automl_run).show()

AutoML_Regression
86722ff2-200e-4e22-ba28-23acac9ea57a


#### Retrieve all Child runs

We use SDK methods to fetch all the child runs and see individual metrics that we log.

In [40]:
children = list(automl_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1
explained_variance,0.39,0.80
mean_absolute_error,4.39,1.59
median_absolute_error,3.48,0.90
normalized_mean_absolute_error,0.02,0.01
normalized_median_absolute_error,0.01,0.00
normalized_root_mean_squared_error,0.03,0.02
normalized_root_mean_squared_log_error,0.10,0.05
r2_score,0.39,0.80
root_mean_squared_error,7.27,3.95
root_mean_squared_log_error,0.43,0.19


### Retreive the best model

Uncomment the below cell to retrieve the best model

In [ ]:
# best_run, fitted_model = automl_run.get_output()
# print(best_run)
# print(fitted_model)

### Test the model

#### Get test data

Uncomment the below cell to get test data

In [ ]:
# split_step = pipeline_run.find_step_run(testTrainSplitStep.name)[0]

# x_test = fetch_df(split_step, output_split_test_x.name)
# y_test = fetch_df(split_step, output_split_test_y.name)

# display(x_test.keep_columns(columns=["vendor", "pickup_weekday", "pickup_hour", "passengers", "distance"]).head(5))
# display(y_test.rename_columns(column_pairs={"Column1": "cost"}).head(5))

# x_test = x_test.to_pandas_dataframe()
# y_test = y_test.to_pandas_dataframe()

#### Test the best fitted model

Uncomment the below cell to test the best fitted model

In [ ]:
# y_predict = fitted_model.predict(x_test.values)

# y_actual =  y_test.iloc[:,0].values.tolist()

# display(pd.DataFrame({'Actual':y_actual, 'Predicted':y_predict}).head(5))

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(14, 10))
# ax1 = fig.add_subplot(111)

# distance_vals = [x[4] for x in x_test.values]

# ax1.scatter(distance_vals[:100], y_predict[:100], s=18, c='b', marker="s", label='Predicted')
# ax1.scatter(distance_vals[:100], y_actual[:100], s=18, c='r', marker="o", label='Actual')

# ax1.set_xlabel('distance (mi)')
# ax1.set_title('Predicted and Actual Cost/Distance')
# ax1.set_ylabel('Cost ($)')

# plt.legend(loc='upper left', prop={'size': 12})
# plt.rcParams.update({'font.size': 14})
# plt.show()